In [1]:
import pandas as pd

In [2]:
review_json_path = '/Users/lilijiaxin/Desktop/project/unsupervised_learning/yelp_dataset/yelp_academic_dataset_review.json'
business_json_path = '/Users/lilijiaxin/Desktop/project/unsupervised_learning/yelp_dataset/yelp_academic_dataset_business.json'

With a huge file like the Yelp dataset, loading all the data at once will most likely crash the memory of the computer. Luckily, Pandas have an option to load large data by segmenting the file into smaller chunks.

In [3]:
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

business = pd.read_json(business_json_path, lines=True)

In [4]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


In [5]:
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    #print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_review)
reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [6]:
reviews.head()

,user_id,business_id,review_stars,text,date
0,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [7]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8021122 entries, 0 to 8021121
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_id       object
 1   business_id   object
 2   review_stars  int64 
 3   text          object
 4   date          object
dtypes: int64(1), object(4)
memory usage: 306.0+ MB


In [8]:
NV_business = business[business['state']=='NV']

In [9]:
NV_reviews = reviews.merge(NV_business,how='right',on='business_id')
NV_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2802658 entries, 0 to 2802657
Data columns (total 18 columns):
 #   Column        Dtype  
---  ------        -----  
 0   user_id       object 
 1   business_id   object 
 2   review_stars  int64  
 3   text          object 
 4   date          object 
 5   name          object 
 6   address       object 
 7   city          object 
 8   state         object 
 9   postal_code   object 
 10  latitude      float64
 11  longitude     float64
 12  stars         float64
 13  review_count  int64  
 14  is_open       int64  
 15  attributes    object 
 16  categories    object 
 17  hours         object 
dtypes: float64(3), int64(3), object(12)
memory usage: 406.3+ MB


In [10]:
NV_reviews.tail()

,user_id,business_id,review_stars,text,date,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
2802653,oyXVHUjz4on1nqMdVuqNNQ,olKAICGTXowdLGVComYJ_g,1,Absolutely horrible management. I applied for ...,2019-08-19 23:28:13,Accent on Rainbow,6666 West Washington Ave,Las Vegas,NV,89107,36.182538,-115.240214,1.5,7,1,None,"Home Services, Real Estate, Apartments","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
2802654,RmttzfOmh7ftdhgrEz-whg,zf2-GEQ32-tN-sxUn3Te6A,5,"Steve from MCS Plumbing was on time, fast, pro...",2019-10-11 23:25:47,MCS plumbing,2383 Winter Cliffs St,Henderson,NV,89052,35.988448,-115.091844,5.0,4,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Plumbing, Home Services","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."
2802655,p0ndG4QEa0D502O9D07hmA,zf2-GEQ32-tN-sxUn3Te6A,5,Our main water pipe had a leak by the meter bo...,2019-12-11 21:40:24,MCS plumbing,2383 Winter Cliffs St,Henderson,NV,89052,35.988448,-115.091844,5.0,4,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Plumbing, Home Services","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."
2802656,wQJ3mPQcrFA71lp2jP85gQ,zf2-GEQ32-tN-sxUn3Te6A,5,We had a great experience with MCS Plumbing on...,2018-10-17 15:26:23,MCS plumbing,2383 Winter Cliffs St,Henderson,NV,89052,35.988448,-115.091844,5.0,4,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Plumbing, Home Services","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."
2802657,PcfjPDJjlAbFTHyxDys8YA,zf2-GEQ32-tN-sxUn3Te6A,5,We hired Steve and his team to help with remod...,2019-07-24 01:50:00,MCS plumbing,2383 Winter Cliffs St,Henderson,NV,89052,35.988448,-115.091844,5.0,4,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Plumbing, Home Services","{'Monday': '7:0-19:0', 'Tuesday': '7:0-19:0', ..."


In [11]:
NV_reviews.isnull().sum()

user_id              0
business_id          0
review_stars         0
text                 0
date                 0
name                 0
address              0
city                 0
state                0
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       67660
categories         423
hours           141685
dtype: int64

In [12]:
reviews = NV_reviews[NV_reviews['is_open']==1]
category_split = reviews['categories'].str.split(',', expand= True)[[0,1,2]]
# select data only in resturant category
category_split.columns = ['category1','category2','category3']
reviews = pd.concat([reviews, category_split],axis=1)
reviews = reviews.drop(['categories'],axis=1)
reviews = reviews[(reviews['category1']=='Restaurants')|(reviews['category2']=='Restaurants')|(reviews['category3']=='Restaurants')]

In [13]:
csv_name = "NV_reviews_business_final.csv"
reviews.to_csv(csv_name, index=False)